In [3]:
import numpy as np
import pandas as pd
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
import uuid
import random
from datetime import datetime, timedelta

In [ ]:
RIDER_DETOUR_PROBS = [0.3, 0.25, 0.2, 0.1, 0.1, 0.05]
DRIVER_DETOUR_PROBS = [0.3, 0.25, 0.2, 0.1, 0.1, 0.05]
RIDER_RATIO = 0.5

In [19]:
df = pd.read_csv("C:/Users/mariam/Downloads/Datasets/New folder/Taxi Trips Chicago 2024/Taxi_Trips_-_2024_20240408.csv")
df.head()
print(df.shape)

(865247, 23)


In [20]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,0287f53fedcde6b0af9aab9e32cbd2cadb337eaa,e54db25f18193a08f1f5754515e8c338480e04fb938ed3...,03/01/2024 12:00:00 AM,03/01/2024 12:00:00 AM,15.0,0.09,NaN,NaN,8.0,8.0,...,35.0,38.75,Credit Card,City Service,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
1,1f0034299b914fd7b45002568576a353c83acb2f,4a263f78f56815be4acd38658af8fc1824ce4e15f7ec81...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,900.0,3.00,NaN,NaN,24.0,8.0,...,0.0,12.00,Cash,Taxi Affiliation Services,41.901207,-87.676356,POINT (-87.6763559892 41.9012069941),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
2,01a0b77b722a0b91f45cb4fc90b3e64f76fd0681,4f1e94982e6851725add382f7981d64006ae7c38f3664f...,03/01/2024 12:00:00 AM,03/01/2024 12:00:00 AM,711.0,5.84,NaN,NaN,56.0,NaN,...,5.0,26.70,Credit Card,City Service,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603),NaN,NaN,NaN
3,14aff2071ac81c6450a8f8b0e1534497da900197,de8e8659ceb9eb0da842a46b60c0a5207098ac69fc23ab...,03/01/2024 12:00:00 AM,03/01/2024 12:30:00 AM,1770.0,13.36,NaN,NaN,76.0,3.0,...,4.0,47.10,Credit Card,Sun Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
4,16d3a633e2f380cee4a7fa4677db163b6e91255d,d79d3e19a1d5f6cb0cd4449d5579412262a2f1b182799d...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,849.0,6.13,NaN,NaN,8.0,3.0,...,0.0,22.82,Mobile,5 Star Taxi,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)


In [ ]:
# Add this to your imports section
import os
from dotenv import load_dotenv
import json
from typing import Dict, Any, Optional

# Define a central configuration section with mappings
def create_column_mapping():
    """Create a configurable mapping between dataset columns and database fields"""
    
    # Default mapping for typical taxi/rideshare datasets
    default_mapping = {
        # Source dataset column : Database column
        "start_time_local": "departure_time",
        "end_time_local": "estimated_arrival_time",
        "pickup_location_latitude": "source_latitude",
        "pickup_location_longitude": "source_longitude",
        "dropoff_location_latitude": "destination_latitude",
        "dropoff_location_longitude": "destination_longitude",
        "trip_distance_meters": "trip_distance_meters"
    }
    
    # You could also load mappings from a config file
    mapping_file = os.getenv('COLUMN_MAPPING_FILE')
    
    if mapping_file and os.path.exists(mapping_file):
        try:
            with open(mapping_file, 'r') as f:
                return json.load(f)
        except Exception as e:
            print(f"Error loading column mapping from {mapping_file}: {e}")
            print("Using default mapping instead.")
    
    return default_mapping

# Dataset-specific mappings
DATASET_MAPPINGS = {
    "san_francisco": {
        "start_time_local": "departure_time",
        "end_time_local": "estimated_arrival_time",
        "pickup_location_latitude": "source_latitude",
        "pickup_location_longitude": "source_longitude",
        "dropoff_location_latitude": "destination_latitude",
        "dropoff_location_longitude": "destination_longitude",
        "trip_distance_meters": "trip_distance_meters"
    },
    "new_york": {
        "pickup_datetime": "departure_time",
        "dropoff_datetime": "estimated_arrival_time",
        "pickup_latitude": "source_latitude",
        "pickup_longitude": "source_longitude",
        "dropoff_latitude": "destination_latitude", 
        "dropoff_longitude": "destination_longitude",
        "trip_distance": "trip_distance_meters"  # Might need conversion if not in meters
    },
    # Add more dataset mappings as needed
}

def load_dataset(file_path, dataset_type=None):
    """
    Load and map a dataset from CSV file using configurable column mappings
    
    Args:
        file_path: Path to the dataset file
        dataset_type: Optional dataset type identifier to use specific mapping
    """
    df = pd.read_csv(file_path)
    
    # Get appropriate mapping for this dataset
    if dataset_type and dataset_type in DATASET_MAPPINGS:
        mapping = DATASET_MAPPINGS[dataset_type]
    else:
        # Try to infer the dataset type from the filename
        filename = os.path.basename(file_path).lower()
        
        if "francisco" in filename or "sf" in filename:
            mapping = DATASET_MAPPINGS["san_francisco"]
        elif "york" in filename or "nyc" in filename:
            mapping = DATASET_MAPPINGS["new_york"]
        else:
            # Use the default mapping
            mapping = create_column_mapping()
    
    # Verify all required columns are present
    missing_columns = [col for col in mapping.keys() if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns in dataset: {missing_columns}")
    
    # Create a new DataFrame with mapped column names
    mapped_df = pd.DataFrame()
    
    for source_col, target_col in mapping.items():
        mapped_df[target_col] = df[source_col]
    
    # Handle special conversions if needed
    # For example, if distance is in miles instead of meters
    if dataset_type == "new_york" and "trip_distance_meters" in mapped_df:
        # Convert miles to meters (1 mile ≈ 1609 meters)
        mapped_df["trip_distance_meters"] = df["trip_distance"] * 1609.34
    
    return mapped_df

In [ ]:
def load_dataset(file_path):
    """Load and filter a dataset from CSV file"""
    df = pd.read_csv(file_path)
    
    columns_to_keep = [
        'start_time_local',
        'end_time_local',
        'pickup_location_latitude',
        'pickup_location_longitude',
        'dropoff_location_latitude',
        'dropoff_location_longitude',
        'trip_distance_meters'
    ]
    
    return df[columns_to_keep]

def connect_to_database():
    """Connect to the database using environment variables"""
    # Load environment variables
    load_dotenv()
    
    # Database connection parameters
    db_params = {
        'host': os.getenv('DB_HOST', 'localhost'),
        'database': os.getenv('DB_NAME', 'matching_engine'),
        'user': os.getenv('DB_USER', 'postgres'),
        'password': os.getenv('DB_PASSWORD'),
        'port': os.getenv('DB_PORT', '5432')
    }
    
    # Check if password is available
    if not db_params['password']:
        raise ValueError("DB_PASSWORD environment variable is not set")
    
    # Connect to database and return connection and cursor
    conn = psycopg2.connect(**db_params)
    return conn


In [ ]:
def meters_to_minutes(meters):
    """Convert trip distance in meters to estimated duration in minutes"""
    # Assuming average speed of 30 km/h = 500 meters per minute
    return max(1, int(meters / 500))

def generate_detour_duration_minutes(weights):
    values = [0, 10, 15, 20, 25, 30]
    return random.choices(values, weights=weights)[0]

def generate_capacity():
    values = [1, 2, 3]
    weights = [0.1, 0.4, 0.35, 0.15]
    return random.choices(values, weights=weights)[0]

def generate_max_walking_duration_minutes():
    values = [5, 10, 15, 20, 25]
    weights = [0.3, 0.35, 0.2, 0.1, 0.05]
    return random.choices(values, weights=weights)[0]

def generate_number_of_riders_per_request():
    values = [1, 2, 3]
    weights = [0.6, 0.3, 0.1]
    return random.choices(values, weights=weights)[0]

def generate_same_gender_preference(gender):
    weights = {
        'female': [0.65, 0.35],
        'male': [0.15, 0.85]   
    }
    gender_weights = weights.get(gender, [0.3, 0.7])  
    
    return random.choices([True, False], weights=gender_weights)[0]

def generate_gender(driver=False):
    if driver:
        return random.choices(['male', 'female'], weights=[0.7, 0.3])[0]
    else:
        return random.choices(['male', 'female'], weights=[0.45, 0.55])[0]

def generate_max_estimated_arrival_time(distance, detour_duration):
    """Generate a random maximum estimated arrival time"""
    return meters_to_minutes(distance) + detour_duration

In [ ]:
def create_rider_requests(data_chunk, start_idx, dataset_id=None, config=None):
    """Create rider requests from a data chunk"""
    requests = []
    
    for i, row in data_chunk.iterrows():
        # Generate a unique ID with optional dataset prefix
        prefix = f"ds{dataset_id}-" if dataset_id is not None else ""
        request_id = f"{prefix}req-{start_idx + i:06d}"
        user_id = f"{prefix}user-{random.randint(1000, 9999)}"
        
        # Parse timestamps
        earliest_departure = pd.to_datetime(row['start_time_local'])
        latest_arrival = pd.to_datetime(row['end_time_local'])
        
        # First generate gender
        gender = generate_gender(driver=False)
        
        # Then generate preferences that correlate with gender
        prefs = generate_preferences(gender)
        
        # Generate other parameters with appropriate configurations
        num_riders = generate_capacity(is_driver=False)
        max_walking = generate_max_walking_duration_minutes()
        
        requests.append((
            request_id,
            user_id,
            float(row['pickup_location_latitude']),
            float(row['pickup_location_longitude']),
            None,  # source_address
            float(row['dropoff_location_latitude']),
            float(row['dropoff_location_longitude']),
            None,  # destination_address
            earliest_departure,
            latest_arrival,
            max_walking,
            num_riders,
            prefs['same_gender'],
            prefs['allows_smoking'],
            prefs['allows_pets'],
            gender,
            dataset_id  # Add dataset_id as the last column
        ))
    
    return requests

In [ ]:
def create_driver_offers(data_chunk, start_idx, dataset_id=None):
    offers = []
    
    for i, row in data_chunk.iterrows():
        # Generate a unique ID
        prefix = f"ds{dataset_id}-" if dataset_id is not None else ""
        offer_id = f"{prefix}drv-{start_idx + i:06d}"
        user_id = f"{prefix}user-{random.randint(10000, 99999)}"
        
        # Parse timestamps
        departure_time = pd.to_datetime(row['start_time_local'])
        estimated_arrival = pd.to_datetime(row['end_time_local'])
        
        # Calculate maximum estimated arrival (add buffer)
        buffer_minutes = random.randint(5, 30)
        max_arrival = estimated_arrival + timedelta(minutes=buffer_minutes)
        
        # Convert distance to detour minutes
        detour_minutes = meters_to_minutes(row['trip_distance_meters'])
        
        # Random capacity (2-6)
        capacity = random.randint(2, 6)
        
        # Generate random preferences
        prefs = random_preferences()
        gender = random_gender()
        
        offers.append((
            offer_id,
            user_id,
            float(row['pickup_location_latitude']),
            float(row['pickup_location_longitude']),
            None,  # source_address
            float(row['dropoff_location_latitude']),
            float(row['dropoff_location_longitude']),
            None,  # destination_address
            departure_time,
            max_arrival,
            estimated_arrival,
            detour_minutes,
            capacity,
            0,  # current_number_of_requests
            prefs['same_gender'],
            prefs['allows_smoking'],
            prefs['allows_pets'],
            gender,
            dataset_id  # Add dataset_id as the last column
        ))
    
    return offers

In [ ]:
def insert_into_database(df, chunk_size=1000, dataset_id=None, rider_ratio=0.6):
    """Process and insert data into database tables"""
    conn = connect_to_database()
    cursor = conn.cursor()
    
    try:
        # Split data: rider_ratio% for rider requests, the rest for driver offers
        split_point = int(len(df) * rider_ratio)
        rider_data = df.iloc[:split_point].reset_index(drop=True)
        driver_data = df.iloc[split_point:].reset_index(drop=True)
        
        print(f"Creating {len(rider_data)} rider requests and {len(driver_data)} driver offers")
        
        # Insert rider requests
        for chunk_start in range(0, len(rider_data), chunk_size):
            chunk_end = min(chunk_start + chunk_size, len(rider_data))
            chunk = rider_data.iloc[chunk_start:chunk_end]
            
            requests = create_rider_requests(chunk, chunk_start, dataset_id)
            
            # SQL for inserting rider requests
            insert_rider_sql = """
            INSERT INTO rider_requests (
                id, user_id, 
                source_latitude, source_longitude, source_address,
                destination_latitude, destination_longitude, destination_address,
                earliest_departure_time, latest_arrival_time,
                max_walking_duration_minutes, number_of_riders,
                same_gender, allows_smoking, allows_pets, user_gender, dataset_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            execute_batch(cursor, insert_rider_sql, requests)
            conn.commit()
            print(f"Inserted {len(requests)} rider requests")
        
        # Insert driver offers
        for chunk_start in range(0, len(driver_data), chunk_size):
            chunk_end = min(chunk_start + chunk_size, len(driver_data))
            chunk = driver_data.iloc[chunk_start:chunk_end]
            
            offers = create_driver_offers(chunk, chunk_start, dataset_id)
            
            # SQL for inserting driver offers
            # SQL for inserting driver offers
            insert_driver_sql = """
            INSERT INTO driver_offers (
                id, user_id,
                source_latitude, source_longitude, source_address,
                destination_latitude, destination_longitude, destination_address,
                departure_time, max_estimated_arrival_time, estimated_arrival_time,
                detour_duration_minutes, capacity, current_number_of_requests,
                same_gender, allows_smoking, allows_pets, user_gender, dataset_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            
            execute_batch(cursor, insert_driver_sql, offers)
            conn.commit()
            print(f"Inserted {len(offers)} driver offers")
        
        print("Data import completed successfully")
        
    except Exception as e:
        conn.rollback()
        print(f"Error: {e}")
        
    finally:
        cursor.close()
        conn.close()

In [ ]:
def create_multiple_datasets(file_path, dataset_count=3, rows_per_dataset=None):
    """Create multiple distinct datasets in the database"""
    df = load_dataset(file_path)
    
    # If rows_per_dataset is not specified, use the full dataset for each
    if rows_per_dataset is None or rows_per_dataset > len(df):
        rows_per_dataset = len(df)
    
    for dataset_id in range(1, dataset_count + 1):
        print(f"\nCreating dataset #{dataset_id}")
        
        # Sample subset of data
        if len(df) > rows_per_dataset:
            sample_df = df.sample(n=rows_per_dataset)
        else:
            sample_df = df
            
        # Apply time shift for dataset variety (optional)
        # Each dataset shifts forward by a week from the last
        day_shift = timedelta(days=(dataset_id-1) * 7)
        sample_df = sample_df.copy()
        sample_df['start_time_local'] = pd.to_datetime(sample_df['start_time_local']) + day_shift
        sample_df['end_time_local'] = pd.to_datetime(sample_df['end_time_local']) + day_shift
        
        # Insert this dataset
        insert_into_database(sample_df, dataset_id=dataset_id)

In [ ]:
if __name__ == "__main__":
    # Create a single dataset
    # df = load_dataset("C:/Users/mariam/Downloads/Datasets/New folder/san francisco/Taxi_Trips.csv")
    # insert_into_database(df)
    
    # Or create multiple datasets
    create_multiple_datasets(
        "C:/Users/mariam/Downloads/Datasets/New folder/san francisco/Taxi_Trips.csv",
        dataset_count=3,
        rows_per_dataset=5000  # Number of rows per dataset
    )